# Simplest RAG

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI, AuthenticationError
from docx import Document
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# load .env file
load_dotenv()

# get api key from environment
api_key = os.environ["OPENAI_API_KEY"]

# create OpenAI client
def create_client(api_key):
    try:
        client = OpenAI(api_key=api_key)
        client.models.list()
        return client
    except AuthenticationError:
        print("Incorrect API")
    return None

client = create_client(api_key)

### Chunking and embedding of input document

In [ ]:
# set file path
filePath=r"../../data/example.docx"

# read file
doc = Document(filePath)

# declare lists
chunks = []
embeddings = []

# text division
for i in range(0, len(doc.paragraphs)):
    chunk = doc.paragraphs[i].text
    chunks.append(chunk)

# create embeddings
for i in range(0, len(chunks)):
    embedding = client.embeddings.create(
        input = chunks[i],
        model = "text-embedding-3-small"
    )
    embeddings.append(embedding.data[0].embedding)

### Query embedding

In [ ]:
# define user query
user_query = "How many samples I need for to assume the normal distribution of the mean?"

# generate embedding
response = client.embeddings.create(
    input = user_query,
    model = "text-embedding-3-small"
)
query_embedding = response.data[0].embedding

"By default, the length of the embedding vector is 1536 for text-embedding-3-small or 3072 for text-embedding-3-large. To reduce the embedding's dimensions without losing its concept-representing properties, pass in the dimensions parameter." (https://platform.openai.com/docs/guides/embeddings/)

In [ ]:
len(query_embedding)

### Retrieval

In [ ]:
# find most similar id
best_match_id = cosine_similarity(
    np.array(embeddings), np.array(query_embedding).reshape(1,-1)
    ).argmax()

# print most similar text
chunks[best_match_id]

### Answer (Augmented Generation)

In [ ]:
# create a chat completion
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
    {
        "role": "system",
        "content": """Using the information contained in the context,
        give a comprehensive answer to the question.
        Respond only to the question asked, response should be concise and relevant to the question.
        Provide the number of the source document when relevant.
        If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": f"""Context:
    {chunks[best_match_id]}
    ---
    Now here is the question you need to answer.
    
    Question: {user_query}""",
        },
    ],
    max_tokens=300
)

# get and print response
print(response.choices[0].message.content)